In [1]:
# Load TensorBoard notebook extension
%load_ext tensorboard

# Imports

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from deepface import DeepFace

2024-01-23 21:46:27.779485: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 21:46:27.818374: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 21:46:27.818396: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 21:46:27.819520: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 21:46:27.825629: I tensorflow/core/platform/cpu_feature_guar

In [3]:
LOG_DIR = './logs/' + datetime.now().strftime("%Y%m%d-%H%M%S")
FILE_WRITER = tf.summary.create_file_writer(LOG_DIR)

2024-01-23 21:46:30.539902: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 21:46:30.577412: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-23 21:46:30.578847: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

# Dataset

In [4]:
ds, info = tfds.load('lfw_pairs',
                      with_info=True,
                      download=True,
                      as_supervised=False)

In [5]:
bench_ds = ds['train'].batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)
tfds.benchmark(bench_ds, batch_size=32)


************ Summary ************



  0%|          | 0/69 [00:00<?, ?it/s]

Examples/sec (First included) 5736.09 ex/sec (total: 2240 ex, 0.39 sec)
Examples/sec (First only) 487.47 ex/sec (total: 32 ex, 0.07 sec)
Examples/sec (First excluded) 6796.68 ex/sec (total: 2208 ex, 0.32 sec)


,duration,num_examples,avg
first+lasts,0.390510,2240,5736.088396
first,0.065645,32,487.467166
lasts,0.324865,2208,6796.678298


In [6]:
with FILE_WRITER.as_default():
    tmp = np.array([x['img_0'] for x in ds['train'].take(25).as_numpy_iterator()])
    tf.summary.image("Pair 0", tmp, max_outputs=25, step=0)
    tmp = np.array([x['img_1'] for x in ds['train'].take(25).as_numpy_iterator()])
    tf.summary.image("Pair 1", tmp, max_outputs=25, step=0)

TypeError: '_PrefetchDataset' object is not subscriptable

In [ ]:
info.features

FeaturesDict({
    'img_0': Image(shape=(250, 250, 3), dtype=uint8),
    'img_1': Image(shape=(250, 250, 3), dtype=uint8),
    'pair': ClassLabel(shape=(), dtype=int64, num_classes=2),
})

In [ ]:
info

tfds.core.DatasetInfo(
    name='lfw_pairs',
    full_name='lfw_pairs/0.1.0',
    description="""
    TODO(lfw_pairs): Markdown description of that will appear on the catalog page.
    Description is **formatted** as markdown.
    
    It should also contain any processing which has been applied (if any),
    (e.g. corrupted example skipped, images cropped,...):
    """,
    homepage='http://vis-www.cs.umass.edu/lfw',
    data_dir='/home/coder/tensorflow_datasets/lfw_pairs/0.1.0',
    file_format=tfrecord,
    download_size=172.28 MiB,
    dataset_size=87.63 MiB,
    features=FeaturesDict({
        'img_0': Image(shape=(250, 250, 3), dtype=uint8),
        'img_1': Image(shape=(250, 250, 3), dtype=uint8),
        'pair': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=None,
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=1000, num_shards=1>,
        'train': <SplitInfo num_examples=2200, num_shards=1>,
    },
    citation="

# Using DeepFace

In [ ]:
for x in ds['test'].take(1).as_numpy_iterator():
  print(DeepFace.represent(x['img_0']))

2024-01-23 20:25:53.344437: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.
2024-01-23 20:25:54.449793: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-01-23 20:25:54.516775: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


[{'embedding': [0.013010134920477867, -0.015226835384964943, 0.013491435907781124, 0.017786256968975067, 0.0026765442453324795, -4.855915904045105e-06, -0.004542464856058359, 0.021797053515911102, 0.011313242837786674, 0.004416103940457106, 0.007855714298784733, 0.016240635886788368, 0.012072200886905193, -0.0010141661623492837, 0.018588628619909286, 0.0024377992376685143, 0.010003570467233658, 0.009951528161764145, 0.009192533791065216, -0.00795767828822136, 0.0097299600020051, 0.011684855446219444, -0.004467829596251249, 0.002481949282810092, 0.0016617102082818747, 0.005378943867981434, 0.00638225581496954, -0.013565946370363235, 0.02041519246995449, 0.013239364139735699, 0.011870996095240116, 0.0019096678588539362, -0.023498231545090675, 0.01239816751331091, 0.0054072909988462925, 0.030711032450199127, 0.006065292749553919, -0.0063626766204833984, 0.0024971156381070614, 0.02355269156396389, 0.00019517168402671814, -0.004966160282492638, -0.00833979994058609, -0.011360103264451027, -

In [ ]:
model = DeepFace.build_model(model_name='Facenet')

In [ ]:
gt = []
pred = []

models = [
  "VGG-Face", 
  "Facenet", 
  "Facenet512", 
  "OpenFace", 
  "DeepFace", 
  "DeepID", 
  "ArcFace", 
  "Dlib", 
  "SFace",
]

ds_test = ds['test'].batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

for x in ds['train'].take(50).as_numpy_iterator():
  pred.append(DeepFace.verify(x['img_0'], x['img_1'], model_name=models[3], enforce_detection=False)['verified'])
  gt.append(x['pair'])

2024-01-23 20:28:51.147851: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(gt, pred))

0.54
